In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

In [3]:
def get_features(spec_file: str, interp_len: int = 500):
    df = pd.read_csv(spec_file)

    # `sid` column does not exist
    if 'sid' not in df.columns:
        df['sdiff']  = df['freq'] < df['freq'].shift(1, fill_value=0)
        df['sdiff'] = df['sdiff'].astype(int)
        df['sid'] = df['sdiff'].cumsum()

    features_interp = []
    for sid, group in df.groupby('sid'):
        freqs = group['freq'].values
        features = group['power'].values
        new_freq = np.linspace(0, 0.5, interp_len)
        new_feat = np.interp(new_freq, freqs, features)
        features_interp.append(new_feat)

    return np.array(features_interp)

In [ ]:
x_gpt4_orig = get_features('../data/gpt-4/pubmed_gpt-4.original.mistral.fftnorm.txt')
y_gpt4_orig = np.zeros(x_gpt4_orig.shape[0])
# print(x_gpt4_orig.shape, y_gpt4_orig.shape)

x_gpt4_samp = get_features('../data/gpt-4/pubmed_gpt-4.sampled.mistral.fftnorm.txt')
y_gpt4_samp = np.ones(x_gpt4_samp.shape[0])
# print(x_gpt4_samp.shape, y_gpt4_samp.shape)

feature_ratio = 0.15
x_gpt4_orig = x_gpt4_orig[:, :int(feature_ratio * x_gpt4_orig.shape[1])]
x_gpt4_samp = x_gpt4_samp[:, :int(feature_ratio * x_gpt4_samp.shape[1])]

x_gpt4 = np.concatenate([x_gpt4_orig, x_gpt4_samp], axis=0)
y_gpt4 = np.concatenate([y_gpt4_orig, y_gpt4_samp], axis=0)

print(x_gpt4.shape, y_gpt4.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x_gpt4, y_gpt4, test_size=0.2, random_state=42)
# x_train, x_test, y_train, y_test = train_test_split(
#     x_davinci, y_davinci, test_size=0.2, random_state=42)

print('train:', x_train.shape, y_train.shape)
print('test:', x_test.shape, y_test.shape)

In [ ]:
model = make_pipeline(StandardScaler(),
    SVC(gamma='auto', kernel='rbf', C=1))
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

scores = cross_val_score(model, x_gpt4, y_gpt4, cv=5)
print(scores)
print(np.mean(scores))

# Using the lower 15% of spectral features, the SVM model achieved an accuracy of 0.75 on the GPT-4 dataset.

### Use FACE-SO style method

In [ ]:
def compute_SO(data, ref):
    """
    Compute the overlap between data and ref
    """
    data_abs = np.abs(data)
    ref_abs = np.abs(ref)
    intersect = np.amin([data_abs, ref_abs], axis=0)
    roof = np.amax([data_abs, ref_abs], axis=0)
    area_intersect = np.trapz(intersect)
    area_roof = np.trapz(roof)
    return area_intersect / area_roof

def compute_SO2(data, ref):
    """
    Compute the overlap between data and ref; using np.sum
    """
    data_abs = np.abs(data)
    ref_abs = np.abs(ref)
    intersect = np.amin([data_abs, ref_abs], axis=0)
    roof = np.amax([data_abs, ref_abs], axis=0)
    area_intersect = np.sum(intersect)
    area_roof = np.sum(roof)
    return area_intersect / area_roof

#todo: Test SO function from FACE-2

In [ ]:
# Test SO functions
data = np.array(range(0,5))
ref = np.array(range(4,-1,-1))

# plot 
import matplotlib.pyplot as plt
plt.plot(data, )
plt.scatter(range(5), data)
plt.plot(ref, )
plt.scatter(range(5), ref)

In [ ]:
so1 = compute_SO(data, ref)
print(so1)
so2 = compute_SO2(data, ref)
print(so2)

# correct answer is 0.3333

### Try mean value distance as a classifier

In [ ]:
# mean_cl_boot(fft.gpt4.orig[freq < 0.5*lower_ratio, power])
#           y     ymin   ymax
#  1 3.264593 2.917913 3.6123
# mean_cl_boot(fft.gpt4.samp[freq < 0.5*lower_ratio, power])
#           y     ymin     ymax
#  1 4.503291 4.084666 4.976448

pow_max_orig = 3.6123
pow_mean_orig = 3.264593
pow_min_samp = 4.084666
pow_mean_samp = 4.503291

def mean_val_cls(x, y, pow_orig, pow_samp, left_end, right_end):
    """
    x: interpolated spectra
    y: 0 for original (human), 1 for sampled (model)
    left_end: left end of the frequency range, [0, 0.5]
    right_end: right end of the frequency range, [0, 0.5]
    """
    assert x.shape[0] == y.shape[0]
    max_freq = 0.5
    tp, tn, fp, fn = 0, 0, 0, 0
    for i in range(x.shape[0]):
        mean_x = np.mean(x[i, int(left_end/max_freq * x.shape[1]): int(right_end/max_freq * x.shape[1])])
        current_y = y[i] # 0 for orig, 1 for samp
        if current_y == 0:
            if np.abs(mean_x - pow_orig) <= np.abs(mean_x - pow_samp): # closer to orig
                tn += 1
            else: # closer to samp
                fp += 1
        else:
            if np.abs(mean_x - pow_orig) <= np.abs(mean_x - pow_samp): # closer to orig
                fn += 1
            else: # closer to samp
                tp += 1
    return tp, tn, fp, fn

def print_scores(tp, tn, fp, fn):
    acc = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)

    print('acc:', acc)
    print('precision:', precision)
    print('recall:', recall)
    print('f1:', f1)

In [ ]:
# use pow_max_orig, pow_min_samp
tp, tn, fp, fn = mean_val_cls(x_gpt4, y_gpt4, pow_max_orig, pow_min_samp, left_end=0.075, right_end=0.5)
print_scores(tp, tn, fp, fn)

## Best perf so far, better than SVM

In [ ]:
tp, tn, fp, fn = mean_val_cls(x_gpt4, y_gpt4, pow_max_orig, pow_min_samp, lower_ration=0.08)
print_scores(tp, tn, fp, fn)

### Main Experiments

In [21]:
# NLL log zscore 
# x_orig = get_features('../data/gpt-4/pubmed_Ans_gpt-4.original.gpt2.nlllogzs.fftreal.txt')
# x_orig = get_features('../data/gpt-4/pubmed_Ans_gpt-4.original.gpt2xl.nlllogzs.fftreal.txt')
x_orig = get_features('../data/gpt-4/pubmed_QA_gpt-4.original.gpt2xl.nlllogzs.fftnorm.circlemean.txt') # Best acc=.80
# x_orig = get_features('../data/gpt-4/pubmed_Ans_gpt-4.original.gpt2xl.nlllogzs.fftnorm.circlemean.txt')
y_orig = np.zeros(x_orig.shape[0])

# x_samp = get_features('../data/gpt-4/pubmed_Ans_gpt-4.sampled.gpt2.nlllogzs.fftreal.txt')
# x_samp = get_features('../data/gpt-4/pubmed_Ans_gpt-4.sampled.gpt2xl.nlllogzs.fftreal.txt')
x_samp = get_features('../data/gpt-4/pubmed_QA_gpt-4.sampled.gpt2xl.nlllogzs.fftnorm.circlemean.txt') # Best acc=.80
# x_samp = get_features('../data/gpt-4/pubmed_Ans_gpt-4.sampled.gpt2xl.nlllogzs.fftnorm.circlemean.txt')
y_samp = np.ones(x_samp.shape[0])

freq_left_end = 0.0
freq_right_end = 0.5
x_orig = x_orig[:, int(freq_left_end/0.5 * x_orig.shape[1]): int(freq_right_end/0.5 * x_orig.shape[1])]
x_samp = x_samp[:, int(freq_left_end/0.5 * x_samp.shape[1]): int(freq_right_end/0.5 * x_samp.shape[1])]

x = np.concatenate([x_orig, x_samp], axis=0)
y = np.concatenate([y_orig, y_samp], axis=0)

In [22]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42)

print('train:', x_train.shape, y_train.shape)
print('test:', x_test.shape, y_test.shape)

train: (240, 500) (240,)
test: (60, 500) (60,)


In [23]:
model = make_pipeline(StandardScaler(),
                      SelectKBest(k=120),
    SVC(gamma='auto', kernel='rbf', C=1))

scores = cross_val_score(model, x, y, cv=5)
print(scores)
print(np.mean(scores))

# NLL log zscore, gpt2 => 0.70

# NLL log zscore, gpt2xl => 0.713

## Use circular to create features
# NLL log zscore, gpt2xl, circle + mean => 0.803

[0.53333333 0.58333333 0.56666667 0.55       0.48333333]
0.5433333333333333


In [ ]:
# bin voting classifier
def bin_cls(x, y, bin_thresholds, bin_size=0.05, start_from_bin=0):
    """
    x: interpolated spectra
    y: 0 for original (human), 1 for sampled (model)
    bin_thresholds: thresholds for each bin
    """
    assert x.shape[0] == y.shape[0]
    assert isinstance(bin_thresholds, list)
    max_freq = 0.5
    tp, tn, fp, fn = 0, 0, 0, 0
    y_predictions = []
    for i in range(x.shape[0]):
        x_means_bin = []
        bin_count = int(max_freq / bin_size)
        bin_width = x.shape[1] // bin_count
        for j in range(bin_count):
            x_means_bin.append(np.mean(x[i, j*bin_width: (j+1)*bin_width]))
        # make prediction for each bin and majority vote
        y_preds = []
        for i, (thres_orig, thres_samp) in enumerate(bin_thresholds):
            if i < start_from_bin:
                continue
            if np.abs(x_means_bin[i] - thres_orig) < np.abs(x_means_bin[i] - thres_samp):
                y_preds.append(0)
            else:
                y_preds.append(1)
        y_predictions.append(y_preds)
        zero_count = y_preds.count(0)
        one_count = y_preds.count(1)
        if zero_count > one_count:
            y_pred = 0
        elif zero_count < one_count:
            y_pred = 1
        else: # tie
            y_pred = 0 if np.random.uniform() < 0.5 else 1
        
        if y[i] == 0: # y[i] is ground truth: 0 for orig, 1 for samp
            if y_pred == 0:
                tn += 1
            else:
                fp += 1
        else:
            if y_pred == 0:
                fn += 1
            else:
                tp += 1
    return tp, tn, fp, fn, y_predictions